In [1]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import json,geetols
%matplotlib inline
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWtgzh4Ry4QVQRsvNb_x4KXWK3-gbUfiw4jLQ8pb3wJXFBlGAVeC95d38JQ

Successfully saved authorization token.


In [2]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)
    return ee.Feature(geometry, stat).set({'date': img.date().format('dd-MM-YYYY')})
  return reduce_region_function
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
          reducer=ee.Reducer.toList().repeat(prop_names.size()),
          selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)

In [67]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    swi = image.normalizedDifference(['B5', 'B11']).rename('SWI')
    return image.addBands(ndvi).addBands(ndwi).addBands(swi)

def addFWI(image):
    image_bands = image.select(['B3', 'B4', 'B8', 'B11', 'B12']).resample('bilinear').reproject(crs="EPSG:4283", scale=10)
    fwi = image_bands.expression('1.7204 + 171*B2 + 3*B3 -70*B4 -45*B5 - 71*B7',
                                {
                                    'B2':image_bands.select('B3'),
                                    'B3':image_bands.select('B4'),
                                    'B4':image_bands.select('B8'),
                                    'B5':image_bands.select('B11'),
                                    'B7':image_bands.select('B12')
                                })
    swiri = image.normalizedDifference(['B11', 'B12']).rename('SWIRI')
    return(image.addBands(fwi.rename("FWI").addBands(swiri)))
    
def addTC(image):
    # Define array of Tasseled Cap coefficients
    coefficients = ee.Array([
    [  0.3037,  0.2793,  0.4743,  0.5585,  0.5082,  0.1863 ],
    [ -0.2848, -0.2435, -0.5436,  0.7243,  0.0840, -0.1800 ],
    [  0.1509,  0.1973,  0.3279,  0.3406, -0.7112, -0.4572 ],
    [ -0.8242,  0.0849,  0.4392, -0.0580,  0.2012, -0.2768 ],
    [ -0.3280,  0.0549,  0.1075,  0.1855, -0.4357,  0.8085 ],
    [  0.1084, -0.9022,  0.4120,  0.0573, -0.0251,  0.0238 ]
  ])

  # Select bands for use in Tasseled Cap
    image_bands_tc = image.select(['B', 'B3', 'B4', 'B8', 'B11', 'B12'])

  # Create 1-D array image (vector of length 6 for all bands per pixel)
    array_image_1d = image_bands_tc.toArray()

  # Create 2-D array image (6x1 matrix for all bands per pixel) from 1-D array
    array_image_2d = array_image_1d.toArray(1)
    
    components_image = ee.Image(coefficients) \
        .matrixMultiply(array_image_2d) \
        .arrayProject([0]) \
        .arrayFlatten([['brightness', 'greenness', 'wetness', 'fourth', 'fifth', 'sixth']])

    return components_image

In [68]:
# Extent of analysis
ext = ee.Geometry.Polygon([[[143.945, -34.242],[143.945, -34.296],[144.067, -34.296],[144.067, -34.242]]])
# Zones to extract mean index/reflectance values
zones = open('C:\\Users\\Liam\\Downloads\\aoi_v2.geojson')
zones = json.load(zones)
# Image collection
ic = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterMetadata('MGRS_TILE', 'equals', '55HBC')\
            .filterDate('2019-01-01','2023-03-01')\
            .filterBounds(ext)\
            .map(lambda img: img.clip(ext))\
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20))\
            .map(addNDVI).map(addFWI)
ic.size()
# Commment out to export image collection to google drive
# tasks = geetools.batch.Export.imagecollection.toDrive(
#             collection=ic,
#             folder='water',
#             region=ext ,
#             scale=10,
#             verbose=True        )

In [69]:
# Iterate over each feature and return the mean index/reflectance value in a data frame
outdf = 1
for i in range(0,len(zones['features'])-1):
    # Select the zone
    stat_zone = ee.Geometry.Polygon(zones["features"][i]["geometry"]["coordinates"][0])
    cover = zones["features"][i]['properties']['cover']
    ff = zones["features"][i]['properties']['ff']
    fid = zones["features"][i]['properties']['Id']
    # Get the mean value
    reduce_im = create_reduce_region_function(geometry = stat_zone,reducer=ee.Reducer.mean(),scale=10,crs='EPSG:4283')
    idx_stat = ee.FeatureCollection(ic.select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12','NDWI','NDVI','FWI','SWI','MSK_CLDPRB']).map(reduce_im).filter(ee.Filter.neq('NDVI', None)))
    # Convert to data frame
    index_dict = fc_to_dict(idx_stat).getInfo()
    index_df = pd.DataFrame(index_dict)
    index_df['id'] = fid
    index_df['cover'] = cover
    index_df['ff'] = ff
    if type(outdf) == int:
        outdf = index_df
    else:
        outdf = outdf.append(index_df)
outdf.to_csv("response61.csv")
    

In [65]:
ic = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2019-03-28','2020-01-01')\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .filter(ee.Filter.eq('relativeOrbitNumber_start',16))\
            .filterBounds(ext)
            #.map(lambda img: img.clip(ext))\

In [56]:

outdf = 1
for i in range(0,len(zones['features'])-1):
    stat_zone = ee.Geometry.Polygon(zones["features"][i]["geometry"]["coordinates"][0])
    cover = zones["features"][i]['properties']['cover']
    ff = zones["features"][i]['properties']['ff']
    fid = zones["features"][i]['properties']['Id']
    reduce_im = create_reduce_region_function(geometry = stat_zone,reducer=ee.Reducer.mean(),scale=10,crs='EPSG:4283')
    idx_stat = ee.FeatureCollection(ic.select(['VV','VH']).map(reduce_im))
    index_dict = fc_to_dict(idx_stat).getInfo()
    index_df = pd.DataFrame(index_dict)
    index_df['id'] = fid
    index_df['cover'] = cover
    index_df['ff'] = ff
    if type(outdf) == int:
        outdf = index_df
    else:
        outdf = outdf.append(index_df)
outdf.to_csv("response_sen1.csv")

In [277]:
type(outdf)

pandas.core.frame.DataFrame